<a href="https://colab.research.google.com/github/Fanusaez/7506R-2C2022-GRUPO13/blob/main/7506R_TP1_GRUPO13_ENTREGA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabajo Practico 1 : Propiedades en Venta
## Grupo 13
### Integrantes:
- Ivan Manuel Pensado
- Axel Tomas Higa
- Edgardo Francisco Saez 
- Lautaro Barrionuevo
- Nicolás Carreño

# Importe de librerias

In [10]:
import pandas as pd
import numpy as np

# Análisis Exploratorio y Preprocesamiento de Datos

Cargamos dataset sobre el que vamos a trabajar

In [11]:
from google.colab import drive
drive.mount("/gdrive")
ds_properati = pd.read_csv('/gdrive/My Drive/properati_argentina_2021.csv') #Hay que tener una copia del dataset subida al drive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [12]:
ds_properati = pd.read_csv('/gdrive/My Drive/properati_argentina_2021.csv')
ds_trabajo = ds_properati.copy()


Cantidad de filas y columnas del dataset

In [13]:
ds_trabajo.shape

(460154, 20)

Columnas del dataset

In [14]:
ds_trabajo.columns.to_list()

['id',
 'start_date',
 'end_date',
 'created_on',
 'latitud',
 'longitud',
 'place_l2',
 'place_l3',
 'place_l4',
 'place_l5',
 'place_l6',
 'operation',
 'property_type',
 'property_rooms',
 'property_bedrooms',
 'property_surface_total',
 'property_surface_covered',
 'property_price',
 'property_currency',
 'property_title']

Tipo de dato de cada columna

In [15]:
ds_trabajo.dtypes

id                           object
start_date                   object
end_date                     object
created_on                   object
latitud                     float64
longitud                    float64
place_l2                     object
place_l3                     object
place_l4                     object
place_l5                     object
place_l6                    float64
operation                    object
property_type                object
property_rooms              float64
property_bedrooms           float64
property_surface_total      float64
property_surface_covered    float64
property_price              float64
property_currency            object
property_title               object
dtype: object

Porcentaje de NaN por columna

In [16]:
filas_totales=ds_trabajo.shape[0]
print(ds_trabajo.isna().sum()/filas_totales*100)

id                            0.000000
start_date                    0.000000
end_date                      0.000000
created_on                    0.000000
latitud                       8.782712
longitud                      8.782712
place_l2                      0.000000
place_l3                      4.887277
place_l4                     69.788375
place_l5                     99.471916
place_l6                    100.000000
operation                     0.000000
property_type                 0.000000
property_rooms               19.918549
property_bedrooms            25.217862
property_surface_total       13.547856
property_surface_covered      7.005915
property_price                3.911951
property_currency             4.034302
property_title                0.000000
dtype: float64


In [17]:
ds_trabajo.head()

,id,start_date,end_date,created_on,latitud,longitud,place_l2,place_l3,place_l4,place_l5,place_l6,operation,property_type,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,property_price,property_currency,property_title
0,XZq2dFGxBiboHY3Bo/tqlw==,2021-09-09,2022-01-27,2021-09-09,-38.957632,-68.105443,Neuquén,NaN,NaN,NaN,NaN,Venta,Casa,6.0,NaN,NaN,206.0,NaN,NaN,Casa - Capital
1,8afoK+yMH/rnnlYiktwgNA==,2021-09-09,2021-10-23,2021-09-09,-34.888878,-56.092368,Montevideo,NaN,NaN,NaN,NaN,Venta,Casa,4.0,3.0,193.0,193.0,390000.0,USD,Casa - Malvín
2,iVaG5rBB47dd1Ya4BMXREA==,2021-09-09,2022-03-09,2021-09-09,-32.488717,-58.257499,Entre Ríos,NaN,NaN,NaN,NaN,Venta,Casa,NaN,NaN,110.0,45.0,47000.0,USD,Casa - Uruguay
3,QE1QVMWKAHWkd0qT0FtPWA==,2021-09-09,2022-01-27,2021-09-09,-34.900165,-54.808453,Maldonado,NaN,NaN,NaN,NaN,Venta,Casa,NaN,3.0,NaN,180.0,400000.0,USD,Casa - El Chorro
4,tJ5lSk7IysG/71OGhacNZA==,2021-09-09,2022-01-27,2021-09-09,-38.954528,-68.041862,Neuquén,NaN,NaN,NaN,NaN,Venta,PH,3.0,2.0,NaN,98.0,102000.0,USD,PH - Villa Farrell


##Exploración Inicial

Filtro los datos y me quedo unicamente con anuncios de propiedades de tipo vivienda ( Casa, PH y Departamento ) ubicados en Capital Federal, cuyo tipo de operación sea venta y su precio se encuentre en (USD).

In [18]:
ds_exploracion = ds_trabajo[ds_trabajo.place_l2.str.contains('Capital Federal', case=False, na = True)]
ds_exploracion = ds_exploracion[ds_exploracion.property_type.str.contains('Departamento|casa|ph', case=False, na =  True)]
ds_exploracion = ds_exploracion[ds_exploracion.operation.str.contains('venta', case=False, na =  True)]
ds_exploracion = ds_exploracion[ds_exploracion.property_currency.str.contains('USD', case=False, na =  True)]

Elimino columnas innecesarias y miro cantidad de filas/columnas dataset

In [19]:
columnas_a_eliminar = ["id"]
ds_exploracion.drop(columns = columnas_a_eliminar, inplace = True)
ds_exploracion.shape

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


(95502, 19)

In [20]:
ds_exploracion["property_price"].describe()

count    9.431300e+04
mean     2.260984e+05
std      3.185460e+05
min      0.000000e+00
25%      9.500000e+04
50%      1.430000e+05
75%      2.370000e+05
max      2.140000e+07
Name: property_price, dtype: float64